<a href="https://colab.research.google.com/github/glukonatic/mystudy/blob/master/finances_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [129]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


Saving details.csv to details (1).csv
User uploaded file "details.csv" with length 4349 bytes


In [0]:
import pandas as pd 
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [145]:
#@title Example form fields
#@markdown Forms support many types of fields.

startdate = '2020-04-01'  #@param {type: "date"}
stopdate = '2020-05-01'  #@param {type: "date"}
#@markdown ---
startdate = np.datetime64(startdate)
stopdate = np.datetime64(stopdate)
data = pd.read_csv(fn, delimiter=';', encoding='cp1251',skiprows=11) 
columns_to_datetime = 'Дата операции'
columns_to_numeric = 'Сумма пересчитанная в валюту счета'
data[columns_to_datetime] = data[columns_to_datetime].apply(pd.to_datetime)
data[columns_to_numeric] = data[columns_to_numeric].str.replace(',','.')
data[columns_to_numeric] = data[columns_to_numeric].str.replace(' ','')
data[columns_to_numeric] = pd.to_numeric(data[columns_to_numeric])
mask_date = (data['Дата операции'] >= startdate) & (data['Дата операции'] <= stopdate)
mask_debt = data['Сумма пересчитанная в валюту счета'] < 0
data_to_plot = data[mask_date][mask_debt][['Дата операции','Сумма пересчитанная в валюту счета','Основание','Номер карты/счета/договора']]
#data_to_plot = data[mask_debt][['Дата операции','Сумма пересчитанная в валюту счета','Основание','Номер карты/счета/договора']]
#sum(data[mask_date][mask_debt]['Сумма пересчитанная в валюту счета'])
data_to_plot['Сумма пересчитанная в валюту счета'] = abs(data_to_plot['Сумма пересчитанная в валюту счета'])
#print(data_to_plot[['Дата операции','Сумма пересчитанная в валюту счета']])
levels = ['Дата операции','Основание','Номер карты/счета/договора']
value_column = 'Сумма пересчитанная в валюту счета'

def build_hierarchical_dataframe(data, levels, value_column, color_columns=None):
    """
    Build a hierarchy of levels for Sunburst or Treemap charts.

    Levels are given starting from the bottom to the top of the hierarchy,
    ie the last level corresponds to the root.
    """
    df_all_trees = pd.DataFrame(columns=['id', 'parent', 'value'])
    for i, level in enumerate(levels):
        df_tree = pd.DataFrame(columns=['id', 'parent', 'value'])
        dfg = data.groupby(levels[i:]).sum(numeric_only=True)
        dfg = dfg.reset_index()
        df_tree['id'] = dfg[level].copy()
        if i < len(levels) - 1:
            df_tree['parent'] = dfg[levels[i+1]].copy()
        else:
            df_tree['parent'] = 'total'
        df_tree['value'] = dfg[value_column]
        df_all_trees = df_all_trees.append(df_tree, ignore_index=True)
    total = pd.Series(dict(id='total', parent='',
                              value=data[value_column].sum()))
    df_all_trees = df_all_trees.append(total, ignore_index=True)
    return df_all_trees


df_all_trees = build_hierarchical_dataframe(data_to_plot, levels, value_column)
'''print(df_all_trees)'''
#average_score = df['sales'].sum() / df['calls'].sum()

fig = make_subplots(1, 2, specs=[[{"type": "domain"}, {"type": "domain"}]],)

fig.add_trace(go.Sunburst(
    labels=df_all_trees['id'],
    parents=df_all_trees['parent'],
    values=df_all_trees['value'],
    branchvalues='total',
    #marker=dict(cmid=average_score),
    hovertemplate='<b>%{label} </b> <br> Sales: %{value}',
    name=''
    ), 1, 1)

fig.add_trace(go.Sunburst(
    labels=df_all_trees['id'],
    parents=df_all_trees['parent'],
    values=df_all_trees['value'],
    branchvalues='total',
    #marker=dict(cmid=average_score),
    hovertemplate='<b>%{label} </b> <br> Sales: %{value}',
    maxdepth=2
    ), 1, 2)

fig.update_layout(margin=dict(t=10, b=10, r=10, l=10))
fig.show()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

